In [1]:
import matplotlib.pyplot as plt 
import numpy as np
import os 
import random
import sys
import pandas as pd
import csv
import re
import time

In [2]:
#read in CSV for  for full corpus
corpus = pd.read_csv('data/interim/extended_podcast_sentences.csv')

print(f'Loaded {corpus.shape[0]} sentences')

Loaded 34357137 sentences


In [18]:
# load target lines
selected = pd.read_csv('data/podcast_sentences_high_conf_5000_with_context.csv')

In [19]:
#print(selected.head())
indexes = selected.iloc[:,0]
print(indexes.head())

0     3967281
1    29184884
2    29661525
3    31833462
4    17157478
Name: Unnamed: 0, dtype: int64


In [21]:
# Loop through all rows to find sentences that were split.

# This delimiter is found it tagging website to for a line break for the UI
UI_delimiter = "--"


rejoined = 0
for (row, i) in enumerate(indexes):
    
    prior_prior_string = corpus.string[i-2]
    prior_prior_source = corpus.source[i-2]
    prior_string = corpus.string[i-1]
    prior_source = corpus.source[i-1]
    selected_string = corpus.string[i]
    selected_source = corpus.source[i]
    post_string = corpus.string[i+1]
    post_source = corpus.source[i+1]
    post_post_string = corpus.string[i+2]
    post_post_source = corpus.source[i+2]
    conf = corpus.confidence[i]
    joined = 1
    context = ""
    if (prior_prior_source == selected_source):
        context += prior_prior_string
        
    if (prior_source == selected_source):
        if (prior_string[-1] not in  ['.','!','?']):
            selected_string = prior_string + " " + selected_string
            conf += corpus.confidence[i-1]
            joined += 1
    else:
        context += " " + prior_source
    
    
    if (post_source == post_source):
        if (selected_string[-1] not in  ['.','!','?']):
            selected_string = selected_string + " " + post_string
            conf += corpus.confidence[i+1]
            joined += 1
            context += " <b> " + selected_string +" </b> "
        else:
            context+= " <b> " + selected_string +" </b> " + post_string
            
    if (post_post_source == selected_source):
        context += " " + post_post_string
    
    # overwrite values in this row with updated versions
    if (joined > 1):
        rejoined += 1
        selected.at[row,"confidence"] = conf/joined
        selected.at[row,"string"] = selected_string
        
    # always fix the contexts
    selected.at[row,"context"] = context    
    selected.at[row,"tagging-context"] = UI_delimiter + context + UI_delimiter + str(i) 
    #print(f'updated selected row {row}: {selected.iloc[row]}')
        
        

#print(outputDf)
print(f'Rejoined {rejoined} strings'  )    

Rejoined 802 strings


In [22]:
# output dataframe to file
selected.to_csv('data/rejoined_selected_podcast_5000_sentences.csv', sep=',',index=False)

In [34]:
# output dataframe to file
tc = selected["tagging-context"]
tc.iloc[1000:4999].to_csv('data/rejoined_selected_podcast_5000_sentences_untagged_context_only.csv', sep=',',index=False)